In [83]:
from utils import *

%matplotlib widget

# libraries for Dimensionality Reduction
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.manifold import Isomap

# libraries for features selectiona
# ANOVA F-test
from sklearn.feature_selection import f_classif
from sklearn import datasets

# library for standardize features
from sklearn.preprocessing import StandardScaler

# libraries for epsilon parameter
from sklearn.neighbors import KNeighborsClassifier

# libraries for Clustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

# libraries for Adjusted Rand Index
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import silhouette_score

# libraries for general utilities
import numpy as np 
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from pandas.plotting import scatter_matrix
from pandas import DataFrame

# global variables from work specifications
SEED = 42
set_seed(SEED)
NUMBER_FEATURES = 6
NEIGHBOURS = 5
MARGIN = 15
i = 0

In [12]:
##
## DATASET LOADING
##

# create the loader class and load dataset and labels
loader = Loader().load()

# get dataset, labels, and dataset splitted by classes
dataset = loader.getDataset()
labels = loader.getLabels()
class_split_dataset = loader.getClassSplitDataset()

In [13]:
dataset.shape

(563, 2500)

In [14]:
##
## FEATURES CREATION
##

# extract 6 features wirh PCA
pca = PCA(n_components=NUMBER_FEATURES)
pca_dataset_embedded = pca.fit_transform(dataset)

# extract 6 features wirh t-sne
tsne = TSNE(n_components=NUMBER_FEATURES, method='exact')
tsne_dataset_embedded = tsne.fit_transform(dataset)

# extract 6 features wirh isomap
isomap = Isomap(n_components=NUMBER_FEATURES)
isomap_dataset_embedded = isomap.fit_transform(dataset)

# features concatenate (6,6,6) = 18 features
features = np.concatenate((pca_dataset_embedded,tsne_dataset_embedded, isomap_dataset_embedded), axis=1)

In [15]:
pca_dataset_embedded.shape

(563, 6)

In [76]:
# standardization
scaler = StandardScaler()
features_std = scaler.fit_transform(features)
ids = labels[:,0]
labels = labels[:,1]

IndexError: too many indices for array

In [17]:
print(features_std.shape)
print(labels.shape)

(563, 18)
(563,)


In [77]:
ids

array([250., 521., 268.,  55., 549., 328., 245., 520.,  82., 529., 257.,
        84., 238., 431., 447.,  70.,  73.,  72., 467., 117., 184., 415.,
       490., 247., 163., 289.,  10., 426., 208., 248.,  76., 355.,   6.,
       500., 101.,  81., 551., 310., 278., 235.,  79.,  86., 365.,  30.,
       377., 438.,  90., 383., 264., 329.,  39., 534., 558.,   2., 144.,
       382., 472., 456., 104., 346., 145., 403., 541., 373., 304., 487.,
       481., 493.,  83., 501., 556., 559., 167., 209.,  78., 176., 274.,
       188.,   9., 118., 465., 185., 109., 218., 380., 140., 192., 137.,
       396.,  77., 400., 522., 153., 376., 381., 443.,  75., 204., 405.,
       449.,  68., 305.,  63., 525., 473.,  15., 132., 527.,  88., 320.,
       357.,  33., 244., 324., 477.,   0.,  11., 290.,  22., 388., 227.,
       131., 148.,  89., 552., 195., 165.,  18., 158., 374., 502., 394.,
       353., 544., 352.,  54., 182.,  46.,  93., 436., 255., 275., 562.,
       425., 384., 177., 335., 497., 348., 452., 44

In [18]:
print(features.shape)
print(labels.shape)

(563, 18)
(563,)


In [19]:
# #############################################################################
# Univariate feature selection with F-test for feature scoring
# We use the default selection function to select the four
# most significant features
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

fig = plt.figure(i, figsize=(8, 6))
i = i+1
feat = features[labels!=0]
lbs = labels[labels!=0]

k_selection = 'all'

# Some noisy data not correlated
#E = np.random.RandomState(42).uniform(0, 0.1, size=(feat.shape[0], 20))

# Add the noisy data to the informative features
#feat = np.hstack((feat, E))

# Split dataset to select feature and evaluate the classifier
X_train, X_test, y_train, y_test = train_test_split(
        feat, lbs, stratify=lbs, random_state=4
)

selector = SelectKBest(f_classif, k=k_selection)
selector.fit(X_train, y_train)
scores = -np.log10(selector.pvalues_)
scores /= scores.sum()
X_indices = np.arange(feat.shape[-1])
plt.bar(X_indices, scores, width=.2,
        label=r'Univariate score ($-Log(p_{value})$)', color='darkorange')#,edgecolor='black')

# #############################################################################
# Compare to the weights of an SVM
clf = make_pipeline(MinMaxScaler(), LinearSVC())
clf.fit(X_train, y_train)
print('Classification accuracy without selecting features: {:.3f}'
      .format(clf.score(X_test, y_test)))

svm_weights = np.abs(clf[-1].coef_).sum(axis=0)
svm_weights /= svm_weights.sum()

plt.bar(X_indices + .2, svm_weights, width=.2, label='SVM weight',
        color='navy')#,edgecolor='black')

# #############################################################################
# SVM on Normalized input

clf_selected = make_pipeline(
        SelectKBest(f_classif, k=k_selection), #MinMaxScaler(), 
        Normalizer(),
    LinearSVC()
)
clf_selected.fit(X_train, y_train)
print('Classification accuracy after selection (Normalized): {:.3f}'
      .format(clf_selected.score(X_test, y_test)))

svm_weights_selected = np.abs(clf_selected[-1].coef_).sum(axis=0)
svm_weights_selected /= svm_weights_selected.sum()

plt.bar(X_indices[selector.get_support()] + .4, svm_weights_selected,
        width=.2, label='SVM selection (Normalized)', color='c')#,edgecolor='black')


# #############################################################################
# SVM on Standardize input

clf_selected = make_pipeline(
        SelectKBest(f_classif, k=k_selection), #MinMaxScaler(), 
        StandardScaler(),
    LinearSVC()
)
clf_selected.fit(X_train, y_train)
print('Classification accuracy after selection (Standardize): {:.3f}'
      .format(clf_selected.score(X_test, y_test)))

svm_weights_selected = np.abs(clf_selected[-1].coef_).sum(axis=0)
svm_weights_selected /= svm_weights_selected.sum()

plt.bar(X_indices[selector.get_support()] + .6, svm_weights_selected,
        width=.2, label='SVM selection (Standardize)', color='pink')#,edgecolor='black')


# #############################################################################
# SVM on MinMax Normalized input

clf_selected = make_pipeline(
        SelectKBest(f_classif, k=k_selection), 
    MinMaxScaler(),
    LinearSVC()
)
clf_selected.fit(X_train, y_train)
print('Classification accuracy after selection (MinMax norm): {:.3f}'
      .format(clf_selected.score(X_test, y_test)))

svm_weights_selected = np.abs(clf_selected[-1].coef_).sum(axis=0)
svm_weights_selected /= svm_weights_selected.sum()

plt.bar(X_indices[selector.get_support()] + .8, svm_weights_selected,
        width=.2, label='SVM selection (MinMax norm)', color='red')#,edgecolor='black')

plt.title("Comparing feature selection")
plt.xlabel('Feature number')
plt.yticks(())
plt.axis('tight')
plt.legend(loc='upper right')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Classification accuracy without selecting features: 0.762
Classification accuracy after selection (Normalized): 0.762
Classification accuracy after selection (Standardize): 0.667
Classification accuracy after selection (MinMax norm): 0.762


In [21]:
corr = abs(DataFrame(features_std).corr())
corr.style.background_gradient(cmap='coolwarm').format("{:.3}")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1.0,6.9e-17,2.15e-17,3.8e-17,7.67e-18,7.69e-18,0.165,0.218,0.416,0.0331,0.333,0.26,0.943,0.145,0.139,0.0152,0.0518,0.0286
1,6.9e-17,1.0,2.47e-16,2.3e-17,5.12e-17,4.49e-18,0.252,0.157,0.0647,0.0853,0.123,0.0296,0.207,0.61,0.573,0.23,0.0886,0.0434
2,2.15e-17,2.47e-16,1.0,3.94e-17,1.09e-17,4.55e-17,0.179,0.0848,0.0603,0.0292,0.119,0.21,0.0185,0.658,0.578,0.117,0.182,0.0338
3,3.8e-17,2.3e-17,3.94e-17,1.0,4.09e-17,3.34e-16,0.116,0.0498,0.0871,0.0837,0.00982,0.0655,0.00618,0.0926,0.173,0.728,0.351,0.139
4,7.67e-18,5.12e-17,1.09e-17,4.09e-17,1.0,2.51e-16,0.00432,0.0337,0.0894,0.0148,0.0282,0.0574,0.0571,0.0288,0.287,0.29,0.705,0.155
5,7.69e-18,4.49e-18,4.55e-17,3.34e-16,2.51e-16,1.0,0.0776,0.0532,0.0284,0.00997,0.032,0.155,0.0178,0.0116,0.0117,0.287,0.0282,0.278
6,0.165,0.252,0.179,0.116,0.00432,0.0776,1.0,0.168,0.267,0.00294,0.261,0.0275,0.192,0.272,0.0638,0.0177,0.0238,0.0287
7,0.218,0.157,0.0848,0.0498,0.0337,0.0532,0.168,1.0,0.02,0.281,0.0931,0.211,0.244,0.021,0.128,0.0218,0.042,0.0866
8,0.416,0.0647,0.0603,0.0871,0.0894,0.0284,0.267,0.02,1.0,0.145,0.138,0.178,0.447,0.00592,0.073,0.0144,0.148,0.0165
9,0.0331,0.0853,0.0292,0.0837,0.0148,0.00997,0.00294,0.281,0.145,1.0,0.118,0.0691,0.0233,0.126,0.0612,0.0116,0.0389,0.0571


In [98]:
(xs, ys) = np.where((corr>0.6)==True)
pairs = [ [x, y] for x,y in zip(xs, ys) if x>y ]
indexes = np.array([int(x) for x in np.union1d(np.ravel(pairs),[])])

In [99]:
pairs

[[12, 0], [13, 1], [13, 2], [15, 3], [16, 4]]

In [100]:
indexes

array([ 0,  1,  2,  3,  4, 12, 13, 15, 16])

In [25]:
scatter_matrix(DataFrame(features_std[:,indexes], columns=indexes), alpha=0.5, diagonal='kde');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
##
## FEATURES REMOVING
##

b = np.arange(features_std.shape[1])
b = np.setdiff1d(b,np.array(pairs)[:,0])

features_sel = features_std[:,b]

In [27]:
b

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 14, 17])

In [28]:
##
## FEATURES EXTRACTION
##
from scipy.stats import f as f_func


# selection with ANOVA F-test
f,prob = f_classif(features_sel, labels)
indexes = np.arange(len(f))
app = [ [i, ele] for i, ele in enumerate(f)]
app.sort(key = lambda app: app[1], reverse = True)
indexes = np.array([ int(a) for a in np.array(app)[:,0]])
print(indexes)
f = f[indexes]
print(np.round(f, 3))
prob = prob[indexes]
print(np.round(prob, 3))

alpha = 0.05
p = len(indexes)
n = len(labels)

critical_value = f_func.ppf(q=1-alpha, dfn=p-1, dfd=n-p)

retire = f>critical_value

print(retire)

[ 2  1  0 12 10  6  4 11  7  5 13  8  3  9]
[32.718 19.248 12.796  5.372  5.209  3.523  3.433  3.134  1.991  1.899
  1.393  0.88   0.632  0.346]
[0.    0.    0.    0.001 0.001 0.015 0.017 0.025 0.114 0.129 0.244 0.451
 0.595 0.792]
[ True  True  True  True  True  True  True  True  True  True False False
 False False]


In [29]:
final_feat = features_sel[:,indexes[retire]]
final_feat.shape

(563, 10)

 # DBSCAN

In [41]:
def findEpsParams(features, title='findEpsParams'):
    
    num_features = features.shape[1]
    
    neigh = KNeighborsClassifier(n_neighbors=NEIGHBOURS)

    nbrs = neigh.fit(features, np.ones(features.shape[0]))
    distances, indices = nbrs.kneighbors(features)

    #print(distances)

    distances = np.sort(distances, axis=0)[:,NEIGHBOURS-1]
    # plt.plot(np.arange(len(distances)), distances, '--', c='red')

    z = np.polyfit(np.arange(len(distances)), distances, 15)
    f2 = np.poly1d(z)
    df2 = np.polyder(f2,1)

    xnew = np.linspace(0, len(distances),features.shape[0], endpoint=True)

    #print(xnew)
    y = f2(xnew)
    ynew = np.array([50*e for e in df2(xnew)])

    #print(ynew.shape)
    x_index = np.where(np.r_[True, ynew[1:] < ynew[:-1]] & np.r_[ynew[:-1] < ynew[1:], True] == True)[0][-3:]
    x_index[-1]=x_index[-1] + MARGIN
    print(x_index)

    plt.axhline(y=y[x_index[0]], linestyle='--', c='blue')
    plt.axhline(y=y[x_index[-1]], linestyle='--', c='red')

    plt.axvline(x=x_index[0], linestyle='--', c='blue')
    plt.axvline(x=x_index[-1], linestyle='--', c='red')

    plt.plot( xnew, y, '-', c='black')

    plt.savefig(f"imgs/{title}_{num_features}_feats.png")
    plt.close()

    return (y[x_index[0]], y[x_index[-1]])

In [90]:
# clustering with DBSCAN
dbscan_by_num = []
eps_matrix = []

for numb in np.arange(2,final_feat.shape[1],1):
    dbscan_by_eps = []
    
    eps_range = findEpsParams(final_feat[:,:numb])
    
    eps_matrix.append(eps_range)
    
    for eps in np.linspace(eps_range[0], eps_range[1], 50):

        dbscan = DBSCAN(eps=eps, min_samples=NEIGHBOURS)
        dbscan_by_eps.append(dbscan.fit_predict(final_feat[:,:numb]))
        
    dbscan_by_num.append(dbscan_by_eps)

[374 468 549]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[372 470 550]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[376 469 550]


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[375 470 547]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[375 470 548]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [85]:
import utilities as uti

dbscan_by_num = np.asarray(dbscan_by_num)
dbscan_by_num.shape

(9, 50, 81)

In [46]:
np.asarray(eps_matrix)

array([[0.01438264, 0.09318756],
       [0.20087879, 0.60515246],
       [0.44547085, 0.92217344],
       [0.64244851, 1.1537154 ],
       [0.75524712, 1.76626679],
       [0.92036725, 2.09295635],
       [1.15727365, 2.28278931],
       [1.31600612, 2.51591068],
       [1.37143368, 2.90621759]])

In [48]:
score_by_num = []

#removing class 0 in the original labels
lab = np.array(labels[labels!=0])
lab = np.array([int(l) for l in lab])


for numb in np.arange(final_feat[labels!=0].shape[1]-start_feat):
    score_by_eps = []
    for eps in np.arange(50):
        # print(numb,eps)
        app_lab = lab
        # app_lab = app_lab[labels!=0]
        
        app_clu = dbscan_by_num[numb][eps]
        
        # change cluster -1 to max+1
        mx = np.max(app_clu)
        mn = np.min(app_clu)
        app_clu[app_clu==mn] = mx+1
        
        
        #removing class -1 point that are "no cluster"
        #app_clu = app_clu[app_clu!=-1]
        #app_lab = app_lab[np.where(app_clu!=-1)]
        
        #(tp, fp, fn, tn) = uti.scores(app_clu, app_lab)
        #randm = uti.rand_index_score(tp, fp, fn, tn) 
        #preci = uti.precision(tp, fp, fn, tn)
        #recal = uti.recall(tp, fp, fn, tn)
        #effe1 = uti.f1(preci, recal)
        # score_by_eps.append([randm, preci, recal, effe1])
        
        adjrn = adjusted_rand_score(app_clu, app_lab)
        score_by_eps.append([adjrn])
        # silue = silhouette_score(app_clu, app_lab, metric = 'euclidean')
        #score_by_eps.append([randm, preci, recal, effe1, adjrn])
        
    score_by_num.append(score_by_eps)
    
score_by_num = np.array(score_by_num)

In [49]:
score_by_num.shape

(9, 50, 1)

In [50]:
np.nanmax(score_by_num, (0,1))

array([0.2739152])

In [51]:
win = np.nanmax(score_by_num, (0,1))
win_ids = []
skip = []
for k in win:
    were = np.where(score_by_num == k)
    if not (np.ravel(were).size/3 >  score_by_num.shape[1]/3):
        win_ids.append(were)
    else:
        win_ids.append(False)
# win_ids = np.delete(np.array(win_ids),2,0)
# print(np.array(win_ids))

win_ids = np.array( [ [ (row[0][j], row[1][j], i) for j, x in enumerate(row[2]) if x==i ] for i, row in enumerate(win_ids) if row != False])
print(win_ids)



[[[4 0 0]]]


In [54]:
win_dic = dict() 
for ele in win_ids:
    for x, y, z in ele:
        if (x,y) not in win_dic:
            win_dic[(x,y)] = set()
        win_dic[(x,y)].add(z)
win_dic

{(4, 0): {0}}

In [97]:
(idi, idj) = list(win_dic.keys())[0]
dbscan_by_num[idi, idj]

array([ 0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  0, -1,  0,  0,  0,  0,  0,
        0, -1,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0, -1,  0,
        0,  0, -1,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0, -1,
        0, -1,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0, -1,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, -1,  0, -1,  0,  0,  0, -1,  0, -1, -1, -1,  0,  0,  0,  0,
        0, -1,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  0,  0,
        0, -1,  0,  1,  0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  0, -1,  0,
        0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0, -1,
       -1, -1,  0,  0, -1,  0,  0,  0,  0,  0,  0, -1, -1,  0,  0,  0, -1,
       -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0, -1, -1,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,
        0,  0, -1,  0,  0

In [62]:
a = eps_matrix[idi]
np.linspace(a[0], a[1], 50)[36]

1.4980370787012247

In [74]:
title = 'cluster'
for x, y in win_dic.keys():

    num_features = x

    a = eps_matrix[idi]
    epsi = np.linspace(a[0], a[1], 50)[36]

    print(win_dic[(x,y)],'\n',dbscan_by_num[x,y])

    ax = Axes3D(fig, elev=-150, azim=110)

    ff = final_feat[labels!=0, :x+1]
    db = dbscan_by_num[x,y]

    #ff = ff[db!=-1,:]
    #db = db[db!=-1]

    print(np.unique(db))

    ax.scatter(ff[:, 0], ff[:, 1], ff[:, 2], c=db,
            cmap=plt.cm.Set1, edgecolor='k', s=40)
    ax.set_title(f"Clustering with DBSCAN [{num_features},{epsi}]")
    ax.set_xlabel("1st feature")
    ax.w_xaxis.set_ticklabels([])
    ax.set_ylabel("2nd feature")
    ax.w_yaxis.set_ticklabels([])
    ax.set_zlabel("3rd feature")
    ax.w_zaxis.set_ticklabels([])

    plt.show()

{0} 
 [2 0 0 0 0 0 0 0 0 0 3 3 3 0 3 3 3 3 3 3 3 3 0 3 3 0 3 0 3 0 0 0 0 0 0 3 0
 0 0 3 3 0 3 0 3 0 3 0 3 0 0 0 2 3 3 2 0 0 3 0 0 0 3 3 0 3 0 0 0 3 3 0 3 0
 3 0 0 0 0 0 3]
[0 2 3]


In [95]:
np.array(dbscan_by_num)[x,y]

array([ 0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  0, -1,  0,  0,  0,  0,  0,
        0, -1,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0, -1,  0,
        0,  0, -1,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0, -1,
        0, -1,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0, -1,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, -1,  0, -1,  0,  0,  0, -1,  0, -1, -1, -1,  0,  0,  0,  0,
        0, -1,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  0,  0,
        0, -1,  0,  1,  0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  0, -1,  0,
        0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0, -1,
       -1, -1,  0,  0, -1,  0,  0,  0,  0,  0,  0, -1, -1,  0,  0,  0, -1,
       -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0, -1, -1,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,
        0,  0, -1,  0,  0

In [80]:
ids[labels!=0].shape

(81,)

In [92]:
from tp2_data.tp2_aux import *

second_file_name= f"htmls/dbscan_by_num_{x},{y}.html"
report_clusters(ids, dbscan_by_num[x,y], second_file_name)

TypeError: list indices must be integers or slices, not tuple